In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the sbg workspace and local utils
import sys
sys.path.append('/home/leandro/Dropbox/workspacesbg/sbg/')
sys.path.append('../../../code/')

from sbg.orf.Orf import OnlineOrf
from sbg.structure.AlphaFoldHandler import AlphaFoldHandler
from sbg.structure.Structure import Structure
from sbg.common.FileHandler import FileHandler
from sbg.scripts.foldx.TangoHandler import TangoHandler

import pipeline_functions as pf

In [3]:
# External libraries
import pandas as pd
import os
from sequence_utils import highlight_sequence, align_sequences_biopython, display_alignment_with_highlighting


In [4]:
# Load the data from data/protein_data.csv
data = pd.read_csv('data/protein_data.csv', index_col=0)

# Computation Pipeline

In [5]:
entry = 0
for up_acc in data.index:
    try:
        entry += 1
        print(f'Processing {up_acc} {data.loc[up_acc, 'swiss_prot']}, {entry} of {len(data.index)}')

        results_dir = f'/home/leandro/Insync/gdrive/Mimark/code/cohort-analysis/notebooks/other_analyses/antigens/data/results/{up_acc[0:2]}/{up_acc}/'
        FileHandler.ensureDir(results_dir)

        # Step 1: Get the Uniprot object
        step = 1
        uniprot_object = OnlineOrf(up_acc)
        step = 1.1
        glycosilation_sites = uniprot_object.getGlycosylationSites()
        if len(uniprot_object.getGlycosylationSites()) > 0:
            glycosylation_df = pd.DataFrame(uniprot_object.getGlycosylationSites(), columns=['res', 'glycosylation'])
            glycosylation_df.to_csv(results_dir + f'{up_acc}_glycosylation.csv')
        
        step = 1.2
        modified_residues = uniprot_object.getModifiedResidues()
        if len(uniprot_object.getModifiedResidues()) > 0:
            modified_residues_df = pd.DataFrame(uniprot_object.getModifiedResidues(), columns=['res', 'modification'])
            modified_residues_df.to_csv(results_dir + f'{up_acc}_modifications.csv')
        
        # step = 1.3
        # try:
        #     isoforms = uniprot_object.getIsoforms()
        # except:
        #     isoforms = []
        # step = 1.4
        # uniprot_string_id = uniprot_object.getStringId()
        # step = 1.5
        # try:
        #     subcellular_location = uniprot_object.getSubcellularLocation()
        # except:
        #     subcellular_location = None
        # step=1.6
        # crytals = uniprot_object.getCrystals()

        # # Step 2: Get the AlphaFold object
        # step = 2
        # alphafold_object = AlphaFoldHandler().get_model(up_acc)

        # # Step 3: Run discotope 3.0 and save the results, then load them
        # step = 3
        # if alphafold_object is not None:
        #     discotope3_file = f'{up_acc}_A_discotope3.csv'
        #     # If no results, run discotope
        #     # if not FileHandler.fileExists(path=results_dir + discotope3_file):
        #     #     pf.run_discotope(up_acc, save_to=results_dir)
            
        #     # Now, if the results are there, load them
        #     if FileHandler.fileExists(path=results_dir + discotope3_file):
        #         discotope_results = pd.read_csv(results_dir + discotope3_file, index_col=0)
        #     else:
        #         discotope_results = None

        # Step 4: Compute the expression 
        step = 4
        exp_file = f'{up_acc}_exp_results.csv'
        #if FileHandler.fileExists(results_dir + exp_file):
        #    exp_results = pd.read_csv(results_dir + exp_file, index_col=0)
        #else:
        try:
            exp_auc, exp_normal, exp_cancer = pf.compare_gene_expression(gene_name=uniprot_object.getGeneName(), up_acc=up_acc, save_to=results_dir)
        except:
            exp_auc, exp_normal, exp_cancer = None, None, None
        
        exp_results = pd.DataFrame({'exp_auc': [exp_auc], 'exp_normal': [exp_normal], 'exp_cancer': [exp_cancer]})
        exp_results.to_csv(results_dir + exp_file)

        # # Step 5: Align the isoforms
        # step = 5
        # if not FileHandler.fileExists(results_dir + f'{up_acc}_isoforms_alignment.html') and len(isoforms) > 1:
        #     secuences = []
        #     names = []
        #     for isoform in isoforms:
        #         isoform_obj = OnlineOrf(isoform)
        #         secuences.append(isoform_obj.getSequence())
        #         names.append(isoform)
        #     alignment = align_sequences_biopython(secuences, names)
        #     display_alignment_with_highlighting(alignment, save_to=results_dir + f'{up_acc}_isoforms_alignment.html')

        # # Step 6: Compute the aggregation propensity
        # step = 6
        # if alphafold_object is not None:
        #     if not FileHandler.fileExists(results_dir + f'{up_acc}_agg.txt'):
        #         TangoHandler.getAggregation(alphafold_object,results_dir)

        #     agg_df = pd.read_csv(results_dir + f'{up_acc}_agg.txt', sep='\t', header=0, index_col="res")[["Aggregation"]]
        
        # # Step 7: Get interaction partners
        # step = 7
        # if not FileHandler.fileExists(results_dir + f'{up_acc}_interactors.tsv'):
        #     interactors = pf.get_interactors(uniprot_string_id, up_acc, save_to=results_dir)
        # else:
        #     interactors = pd.read_csv(results_dir + f'{up_acc}_interactors.tsv', sep='\t', index_col=0)

        # # Step 8: Get the protein homology
        # step = 8
        # if not FileHandler.fileExists(results_dir + f'{up_acc}_homologs.tsv'):
        #     homology = pf.get_homologs(uniprot_object.getGeneName(), up_acc, save_to=results_dir)
        # else:
        #     homology = pd.read_csv(results_dir + f'{up_acc}_homologs.tsv', sep='\t', index_col=0)

        # # Step 9: Get the protein structures bioassemblies to check for homo and multimers
        # step = 9
        # if not FileHandler.fileExists(results_dir + f'{up_acc}_bioassemblies.csv'):
        #     try:
        #         all_dataframes = []
        #         for crystal in crytals:
        #             structure = Structure(crystal, replaceExistent=False)
        #             df = structure.analyzeStructureType()
        #             df['crystal_id'] = crystal
        #             all_dataframes.append(df)

        #         if len(all_dataframes) > 1:
        #             bioassemblies_df = pd.concat(all_dataframes)
        #         elif len(all_dataframes) == 1:
        #             bioassemblies_df = all_dataframes[0]
        #         else:
        #             bioassemblies_df = None

        #         if len(all_dataframes) > 0:
        #             bioassemblies_df.to_csv(results_dir+f'{up_acc}_bioassemblies.csv')
        #     except:
        #         bioassemblies_df = None
        # else:
        #     bioassemblies_df = pd.read_csv(results_dir + f'{up_acc}_bioassemblies.csv')

    except Exception as e:
        print(f'Error with {up_acc} in step {step}: {e}')

Processing P07355 ANXA2_HUMAN, 1 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing O00425 IF2B3_HUMAN, 2 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing O15392 BIRC5_HUMAN, 3 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing P00749 UROK_HUMAN, 4 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing P02787 TRFE_HUMAN, 5 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing P03372 ESR1_HUMAN, 6 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing P04792 HSPB1_HUMAN, 7 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing P07339 CATD_HUMAN, 8 of 505
Not enough samples in one or both groups to perform statistical comparison.
Processing P08253 MMP2_HUMAN, 9 of 505
Not enough samples in one or both gro

# Auxiliary: epitopes

In [ ]:
up_accs = {
    "KPYM": "P14618",
    "MMP9": "P14780",
    "HSPB1": "P04792",
    "AGRIN": "O00468",
    "CLIC1": "O00299"
}

epitopes = {
    "KPYM": ["EAVRMQHLIARE"],
    "MMP9": ["TFLGKEY", "GYPFDGKD"],
    "HSPB1": ["KDGVV"],
    "AGRIN": ["RLELSRHW", "FVGAGLRGC", "NPCHGAAPC", "RDRRLEF", "GHPCLNGASC", "VCLCPGGF"],
    "CLIC1": ["KRRTE", "QVELF", "KRRTET"] 
}


up_objs = {k: OnlineOrf(v) for k, v in up_accs.items()}



In [ ]:
def find_all_epitopes(seq, epitope):
    positions = []
    for i in range(len(seq) - len(epitope)):
        if seq[i:i+len(epitope)] == epitope:
            positions.append((i, i+len(epitope)))
    return positions

for up in up_objs.keys():
    print(f'Processing {up}')
    up_obj = up_objs[up]
    seq = up_obj.getSequence()

    for epitope in epitopes[up]:
        print(f'Epitope: {epitope}')
        print(f'Positions: {find_all_epitopes(seq, epitope)}')

    print('\n\n')


Processing KPYM
Epitope: EAVRMQHLIARE
Positions: [(372, 384)]



Processing MMP9
Epitope: TFLGKEY
Positions: [(351, 358)]
Epitope: GYPFDGKD
Positions: [(177, 185)]



Processing HSPB1
Epitope: KDGVV
Positions: [(113, 118)]



Processing AGRIN
Epitope: RLELSRHW
Positions: [(1455, 1463)]
Epitope: FVGAGLRGC
Positions: [(1510, 1519)]
Epitope: NPCHGAAPC
Positions: [(1594, 1603)]
Epitope: RDRRLEF
Positions: [(1695, 1702)]
Epitope: GHPCLNGASC
Positions: [(1826, 1836)]
Epitope: VCLCPGGF
Positions: [(1843, 1851)]



Processing CLIC1
Epitope: KRRTE
Positions: [(48, 53)]
Epitope: QVELF
Positions: [(6, 11)]
Epitope: KRRTET
Positions: [(48, 54)]



